In [146]:
%load_ext autoreload
%autoreload 2
import numpy as np
from copy import deepcopy
import sys
import datasets
import matplotlib.pyplot as plt
import numpy as np
sys.path.append('../experiments/')
import os
import pandas as pd
import numpy as np
import sys
from os.path import join
import datasets
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from imodelsx import AugLinearClassifier

# load dset
dset = datasets.load_dataset('rotten_tomatoes')['train']
dset = dset.select(np.random.choice(len(dset), size=100, replace=False))

dset_val = datasets.load_dataset('rotten_tomatoes')['validation']
dset_val = dset_val.select(np.random.choice(
    len(dset_val), size=100, replace=False))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [147]:
m = AugLinearClassifier(
    checkpoint='bert-base-uncased',
    ngrams=2,
    all_ngrams=True,  # also use lower-order ngrams
    zeroshot_class_dict={0: ['negative', 'boring'],
                         1: ['positive', 'great']},
    prune_stopwords=True,
)
m.fit(None, [0, 1])
m.cache_linear_coefs(dset_val['text'], renormalize_embs=True)

initializing model...
setting up zero-shot linear model...


100%|██████████| 4/4 [00:22<00:00,  5.60s/it]

	After caching, coefs_dict_ len 1805


In [151]:
from sklearn.metrics import roc_auc_score
preds = m.predict(dset_val['text'])
# m._predict_cached(dset_val['text'], warn=False)
preds_proba = m.predict_proba(dset['text'])
print('acc_val', np.mean(preds == dset_val['label']))
print('roc', roc_auc_score(dset_val['label'], preds_proba[:, 1]).round(2))

[0 0 1 1 0 1 1 1 0 1 1 1 1 0 0 0 1 0 1 0 1 0 1 0 0 1 0 0 0 1 1 0 0 0 1 0 1
 1 0 1 0 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 0 0 0 1 0 0 1 0 1 1 1 1 1 1 1 0 1 1
 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0 0 1 1 0 0 1]
acc_val 0.57
roc 0.54


/home/chansingh/imodelsx/imodelsx/auglinear/auglinear.py:419: UserWarning: Saw an unseen ungram 1633 times. For better performance, call cache_linear_coefs on the test dataset before calling predict.
  warnings.warn(


In [144]:
m._predict_cached(['fun', 'great', 'great positive nice fun'])

array([ 4.4904089 , 15.62479973, 20.11520863])

In [152]:
print('Total ngram coefficients: ', len(m.coefs_dict_))
print('Most positive ngrams')
for k, v in sorted(m.coefs_dict_.items(), key=lambda item: item[1], reverse=True)[:30]:
    print('\t', k, round(v, 2))
print('Most negative ngrams')
for k, v in sorted(m.coefs_dict_.items(), key=lambda item: item[1])[:30]:
    print('\t', k, round(v, 2))

Total ngram coefficients:  1805
Most positive ngrams
	 great 10.0
	 good 8.63
	 enthusiastic 8.57
	 good overall 7.41
	 big 6.79
	 splendid 6.66
	 convincing 6.63
	 mean 6.37
	 bravo 6.19
	 man 6.17
	 obsessed 6.16
	 missed 6.14
	 guys 6.08
	 pretty 6.07
	 sure 6.04
	 scary 6.04
	 spirited 6.02
	 effectiveness 6.0
	 hilarious 6.0
	 fun 5.99
	 predictable 5.98
	 obvious 5.88
	 neat 5.78
	 foul 5.71
	 bigger 5.69
	 quirky 5.67
	 right 5.64
	 spy 5.64
	 rude 5.62
	 stuff 5.62
Most negative ngrams
	 sci -7.72
	 meditation ethereal -7.19
	 parachutes onto -6.97
	 parachutes -6.86
	 moves away -6.77
	 cardellini -6.39
	 benjamins -6.34
	 representation every -6.27
	 society -6.25
	 arm's -6.14
	 preserves tosca -5.91
	 single stroke -5.87
	 atmospheric meditation -5.86
	 onto moving -5.75
	 lillard cardellini -5.74
	 deliberate gait -5.66
	 life arm's -5.65
	 pretensions -5.51
	 signposts discovering -5.33
	 've seen -5.26
	 pena taquilla -5.19
	 cobbled together -5.15
	 sensación inconformi

# Finetuned version

In [ ]:
m = AugLinearClassifier(
    checkpoint='textattack/distilbert-base-uncased-rotten-tomatoes',
    ngrams=2,
    all_ngrams=True,  # also use lower-order ngrams
)
m.fit(dset['text'], dset['label'])
m.cache_linear_coefs(dset_val['text'])

In [ ]:
preds = m.predict(dset['text'])
print('acc_train', np.mean(preds == dset['label']))
preds_proba = m.predict_proba(dset['text'])
preds = m.predict(dset_val['text'])
print('acc_val', np.mean(preds == dset_val['label']))

In [ ]:
print('Total ngram coefficients: ', len(m.coefs_dict_))
print('Most positive ngrams')
for k, v in sorted(m.coefs_dict_.items(), key=lambda item: item[1], reverse=True)[:8]:
    print('\t', k, round(v, 2))
print('Most negative ngrams')
for k, v in sorted(m.coefs_dict_.items(), key=lambda item: item[1])[:8]:
    print('\t', k, round(v, 2))